In [0]:
from fancyimpute import IterativeImputer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
Using TensorFlow backend.


In [0]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded['train.csv']))
df_test = pd.read_csv(io.BytesIO(uploaded['test.csv']))
df = df.drop(['Ticket','Cabin','Name'],axis=1)
df['Gender'] = df['Sex'].map({'male':1,'female':0})
df['Embarked'] = df['Embarked'].map({'S':0,'C':1,'Q':2})

In [0]:
df['Embarked']= df['Embarked'].fillna(0.0)
df.drop(['Sex'],axis=1,inplace=True)
df.Age.isna().sum()/len(df)

0.19865319865319866

In [0]:
train_cols = df.columns
df_1 = pd.DataFrame(IterativeImputer().fit_transform(df))
df_1.columns = train_cols
df_1.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,Gender
0,1.0,0.0,3.0,22.0,1.0,0.0,7.2500,0.0,1.0
1,2.0,1.0,1.0,38.0,1.0,0.0,71.2833,1.0,0.0
2,3.0,1.0,3.0,26.0,0.0,0.0,7.9250,0.0,0.0
3,4.0,1.0,1.0,35.0,1.0,0.0,53.1000,0.0,0.0
4,5.0,0.0,3.0,35.0,0.0,0.0,8.0500,0.0,1.0


In [0]:
df_1.drop(df_1.index[df_1['Age']<0],inplace=True)
df_1[df_1['Fare']>300]
df_1.drop(df_1.index[df_1['Fare']>300],inplace=True)
df_1.set_index(df_1['PassengerId'],inplace=True)
df_1.drop(['PassengerId'],axis=1,inplace=True)

In [0]:
col = ['Pclass','SibSp','Parch','Embarked']
for i in col:
  dummy1 = pd.get_dummies(df_1[i],prefix=i,drop_first=True)
  df_1 = pd.concat([df_1,dummy1],axis=1)
df_y = pd.DataFrame(df_1['Survived'])
df_1.drop(['Survived'],axis=1,inplace=True)
df_1.drop(['Pclass','SibSp','Parch','Embarked'],axis=1,inplace=True)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train , y_test = train_test_split(df_1, df_y, train_size=0.7, test_size=0.3, random_state=100)

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train[['Age','Fare']] = scaler.fit_transform(X_train[['Age','Fare']])
X_train.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Age,Fare,Gender,Pclass_2.0,Pclass_3.0,SibSp_1.0,SibSp_2.0,SibSp_3.0,SibSp_4.0,SibSp_5.0,Parch_1.0,Parch_2.0,Parch_3.0,Parch_4.0,Parch_5.0,Parch_6.0,Embarked_1.0,Embarked_2.0
PassengerId,,,,,,,,,,,,,,,,,,
792.0,-1.008301,-0.098247,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
163.0,-0.262837,-0.558061,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
818.0,0.109895,0.179388,1.0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0
849.0,-0.113744,0.078362,1.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
648.0,1.973556,0.141437,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [0]:
X_test[['Age','Fare']] =  scaler.transform(X_test[['Age','Fare']])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [0]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [6,7,8,9,10],
    'min_samples_leaf': range(50, 400, 50),
    'min_samples_split': range(50, 400, 50),
    'n_estimators': [100,200, 300], 
    'max_features': [5,6,8,9,10,12,14]
}
n_folds = 5
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = n_folds, n_jobs = -1,verbose = 1)

In [0]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 5145 candidates, totalling 25725 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   46.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 14.9min
[Parallel(n_jobs=-1)]: Done 4996 tasks      | elapsed: 18.5min
[Parallel(n_jobs=-1)]: Done 6046 tasks      | elapsed: 22.3min
[Parallel(n_jobs=-1)]: Done 7196 tasks      | elapsed: 26.6min
[Parallel(n_jobs=-1)]: Done 8446 tasks      | elapsed: 31.2min
[Parallel(n_jobs=-1)]: Done 9796 tasks      | elapsed: 36.2min
[Parallel(n_jobs=-1)]: Done 11246 tasks      |

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [0]:
print('We can get accuracy of',grid_search.best_score_,'using',grid_search.best_params_)

We can get accuracy of 0.797077922077922 using {'max_depth': 8, 'max_features': 9, 'min_samples_leaf': 50, 'min_samples_split': 150, 'n_estimators': 200}


In [0]:
# rfc = RandomForestClassifier(bootstrap=True,
#                              max_depth=8,
#                              min_samples_leaf=100, 
#                              min_samples_split=200,
#                              max_features=10,
#                              n_estimators=200)

rfc = RandomForestClassifier(bootstrap=True,
                             max_depth=8,
                             min_samples_leaf=50, 
                             min_samples_split=150,
                             max_features=9,
                             n_estimators=200)

In [0]:
rfc.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features=9, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=50, min_samples_split=150,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
predictions = rfc.predict(X_test)

In [0]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import metrics

In [0]:
print ("Accuracy : %.4g" % metrics.accuracy_score(y_test, predictions))

Accuracy : 0.8


In [0]:
df_test.drop(['Ticket','Name','Cabin'],axis=1,inplace=True)
df_test.set_index(df_test['PassengerId'],drop=True,inplace=True)
df_test.drop(['PassengerId'],axis=1,inplace=True)
df_test['Gender'] = df_test['Sex'].map({'male':1,'female':0})
df_test['Embarked'] = df_test['Embarked'].map({'S':0,'C':1,'Q':2})
df_test.drop(['Sex'],axis=1,inplace=True)
train_cols = df_test.columns
df_test_1 = pd.DataFrame(IterativeImputer().fit_transform(df_test))
df_test_1.columns = train_cols

In [0]:
df_test_1.set_index(df_test.index,drop=True,inplace=True)

In [0]:
df_test_1.Fare[df_test_1['Fare']>280] = df_test_1.Fare.mean()

In [0]:
col = ['Pclass','SibSp','Parch','Embarked']
for i in col:
  dummy1 = pd.get_dummies(df_test_1[i],prefix=i,drop_first=True)
  df_test_1 = pd.concat([df_test_1,dummy1],axis=1)

df_test_1.drop(['Pclass','SibSp','Parch','Embarked'],axis=1,inplace=True)

In [0]:
df_test_1[['Age','Fare']] =  scaler.transform(df_test_1[['Age','Fare']])
df_test_1.drop(['SibSp_8.0','Parch_9.0'],axis=1,inplace=True)
Y_pred = pd.DataFrame(index=df_test_1.index)
Y_pred['Predicted_Value'] = rfc.predict(df_test_1)

In [0]:
df_test_1 = pd.concat([df_test_1,Y_pred],axis=1)
df_test_1.head()

,Age,Fare,Gender,Pclass_2.0,Pclass_3.0,SibSp_1.0,SibSp_2.0,SibSp_3.0,SibSp_4.0,SibSp_5.0,Parch_1.0,Parch_2.0,Parch_3.0,Parch_4.0,Parch_5.0,Parch_6.0,Embarked_1.0,Embarked_2.0,Predicted_Value
PassengerId,,,,,,,,,,,,,,,,,,,
892,0.370808,-0.556694,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0
893,1.302638,-0.577614,0.0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0
894,2.420834,-0.509809,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0
895,-0.188291,-0.535670,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
896,-0.561023,-0.444211,0.0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1.0


In [0]:
from google.colab import files
df_test_1.to_csv('predicted_value_random_forest.csv')
# with open('predicted_value.csv', 'w') as f:
#   f.write(df_test_1)

files.download('predicted_value_random_forest.csv')